In [ ]:
# General imports
import pandas as pd
import os
import numpy as np

# Pandas display settings
pd.set_option("display.max_colwidth",150)
pd.set_option('display.max_columns', None)

In [ ]:
# Main JEM directory
path_main_jem = "//allen/programs/celltypes/workgroups/279/Patch-Seq/compiled-jem-data"

# JEM csv files
path_jem = os.path.join(path_main_jem, "jem_metadata.csv")
path_jem_na = os.path.join(path_main_jem, "NA_jem_metadata.csv")
path_jem_fail = os.path.join(path_main_jem, "jem_metadata_wFAILURE.csv")

In [ ]:
# Read all jem dataframes
jem_df = pd.read_csv(path_jem, low_memory=False)
jem_na_df = pd.read_csv(path_jem_na, low_memory=False)
jem_fail_df = pd.read_csv(path_jem_fail, low_memory=False)

In [ ]:
# Replace values 
jem_df["status"] = jem_df["status"].replace({"SUCCESS (high confidence)": "SUCCESS"})
jem_fail_df["status"] = jem_fail_df["status"].replace({"SUCCESS (high confidence)": "SUCCESS",
                                                       "NO ATTEMPTS": "FAILURE",
                                                       "Failure": "FAILURE"})

# Filter dataframe to only FAILURE
jem_fail_df = jem_fail_df[jem_fail_df["status"] == "FAILURE"]

# Filter tubes and NAs
jem_df = jem_df[(jem_df["status"] == "SUCCESS")&(~jem_df["container"].isnull())]
jem_na_df = jem_na_df[(jem_na_df["status"] == "SUCCESS")&(jem_na_df["container"].isnull())]

# Replace values
jem_na_df["container"] = jem_na_df["container"].replace({np.nan: "Not Applicable"})

# Merging all jem dataframes
final_jem_df = pd.concat([jem_df, jem_na_df, jem_fail_df], ignore_index=True, sort=False)

In [ ]:
jem_dictionary = {
    # date
    "acsfProductionDate": "jem-date_acsf",
    "blankFillDate": "jem-date_blank",
    "internalFillDate": "jem-date_internal",
    "date": "jem-date_patch",
    # depth
    "depth": "jem-depth_current",
    "approach.depth": "jem-depth_old",
    "cell_depth": "lims-depth", #LIMS?
    # health
    "approach.cellHealth": "jem-health_cell",
    "extraction.fillquality": "jem-health_fill_quality",
    "approach.sliceHealth": "jem-health_slice",
    "sliceQuality": "jem-health_slice_initial",
    "sliceQualityFinal": "jem-health_slice_final",
    # id
    "pipetteSpecName": "jem-id_cell_specimen",
    "container": "jem-id_patched_cell_container",
    "limsSpecName": "jem-id_slice_specimen",
    "rigNumber": "jem-id_rig_number",
    "rigOperator": "jem-id_rig_user",
    "name": "lims-id_cell_specimen", #LIMS?
    "specimen_ID": "lims-id_cell_specimen_id", #LIMS?
    "full_genotype": "lims-id_slice_genotype", #LIMS?
    "organism_name": "lims-id_species", #LIMS?
    # notes
    "extraction.extractionNotes": "jem-notes_extraction",
    "freeFailureNotes": "jem-notes_failure",
    "sliceNotes": "jem-notes_overall",
    "qcNotes": "jem-notes_qc",
    # nucleus
    "extraction.endPipetteR": "jem-nucleus_end_seal_res",
    "recording.pipetteR": "jem-nucleus_pipette_res",
    "extraction.postPatch": "jem-nucleus_post_patch",
    "extraction.nucleus": "jem-nucleus_sucked",
    # options
    "attempt": "jem-options_attempt",
    "failureNotes": "jem-options_failure",
    "successNotes": "jem-options_success",
    "virus_enhancer": "jem-options_virus_enhancer",
    # pressure
    "extraction.pressureApplied": "jem-pressure_extraction",
    "extraction.retractionPressureApplied": "jem-pressure_retraction",
    # project
    "approach.pilotName": "jem-project_name",
    "approach.project_retrograde_labeling_hemisphere": "jem-project_retrograde_labeling_hemisphere",
    "approach.project_retrograde_labeling_region": "jem-project_retrograde_labeling_region",
    "approach.project_retrograde_labeling_exp": "jem-project_retrograde_labeling_exp",
    # region
    "roi": "jem-region_major_minor",
    "roi_major": "jem-region_major", 
    "roi_minor": "jem-region_minor",
    # status
    "approach.creCell": "jem-status_reporter",
    "status": "jem-status_success_fail",
    # time
    "recording.timeStart": "jem-time_exp_approach_start",
    "extraction.timeChannelRecordingEnd": "jem-time_exp_channel_end",
    "extraction.timeEnd": "jem-time_exp_end_old", 
    "extraction.timeExtractionEnd": "jem-time_exp_extraction_end",
    "extraction.timeExtractionStart": "jem-time_exp_extraction_start",
    "extraction.timeRetractionEnd": "jem-time_exp_retraction_end_current",
    "recording.timeWholeCellStart": "jem-time_exp_whole_cell_start",
    # test
    "lims_check": "test-mismatch_jem_lims"}

In [ ]:
# Renaming columns based on dictionary
final_jem_df.rename(columns=jem_dictionary, inplace=True)

In [ ]:
# Filtering dataframe to only IVSCC Group 2017-Current
ivscc_rig_users = ["aarono","balreetp","brianle","dijonh","gabrielal","jessicat","katherineb",
                   "kristenh","lindsayn","lisak","ramr","rustym","sarav"]
ivscc_rig_numbers = ["1","2","3","4","5","6","7","8"]
final_jem_df = final_jem_df[final_jem_df["jem-id_rig_user"].isin(ivscc_rig_users)]
final_jem_df = final_jem_df[final_jem_df["jem-id_rig_number"].isin(ivscc_rig_numbers)]

In [ ]:
#Fixing depth/time column and combining into one column
final_jem_v109_df = final_jem_df[final_jem_df["formVersion"] == "1.0.9"]
final_jem_vother_df = final_jem_df[final_jem_df["formVersion"] != "1.0.9"]
final_jem_v109_df.rename(columns={"jem-depth_old": "jem-depth",
                                  "jem-time_exp_end_old": "jem-time_exp_retraction_end"}, 
                         inplace=True)
final_jem_vother_df.rename(columns={"jem-depth_current": "jem-depth", 
                                    "jem-time_exp_retraction_end_current": "jem-time_exp_retraction_end"}, 
                           inplace=True)
final_jem_df = pd.concat([final_jem_v109_df, final_jem_vother_df])

In [ ]:
# Fixing datetime column to only date column
split_date = final_jem_df["jem-date_patch"].str.split(" ", n=1, expand=True) # Splitting date and time into 2 columns
final_jem_df["jem-date_patch"] = split_date[0] # Choosing column with only the dates

# Removing timezones from time columns
columns_time = ["jem-time_exp_approach_start", "jem-time_exp_channel_end", "jem-time_exp_extraction_end", 
                "jem-time_exp_extraction_start", "jem-time_exp_retraction_end", "jem-time_exp_whole_cell_start"]  
for col in columns_time:
    split_timezone = final_jem_df[col].str.split(" ", n=1, expand=True) # Splitting time and timezone into 2 columns
    final_jem_df[col] = split_timezone[0] # Choosing column with only the time

# Adding in year, monnth, day columns
final_jem_df["jem-date_patch_y-m-d"] = final_jem_df["jem-date_patch"]
split_date = final_jem_df["jem-date_patch_y-m-d"].str.split("-", n=2, expand=True) # Splitting year, month and day
final_jem_df["jem-date_patch_y"] = split_date[0] # Choosing column with years
final_jem_df["jem-date_patch_m"] = split_date[1] # Choosing column with months
final_jem_df["jem-date_patch_d"] = split_date[2] # Choosing column with days    

In [ ]:
# Changing date columns to a datetime
final_jem_df["jem-date_acsf"] = pd.to_datetime(final_jem_df["jem-date_acsf"])
final_jem_df["jem-date_blank"] = pd.to_datetime(final_jem_df["jem-date_blank"])
final_jem_df["jem-date_internal"] = pd.to_datetime(final_jem_df["jem-date_internal"])
final_jem_df["jem-date_patch"] = pd.to_datetime(final_jem_df["jem-date_patch"])

# Changing date column format to MM/DD/YYYY
final_jem_df["jem-date_acsf"] = final_jem_df["jem-date_acsf"].dt.strftime("%m/%d/%Y")
final_jem_df["jem-date_blank"] = final_jem_df["jem-date_blank"].dt.strftime("%m/%d/%Y")
final_jem_df["jem-date_internal"] = final_jem_df["jem-date_internal"].dt.strftime("%m/%d/%Y")
final_jem_df["jem-date_patch"] = final_jem_df["jem-date_patch"].dt.strftime("%m/%d/%Y")

# Convert string column to float column and apply absolute value to columns
final_jem_df["jem-depth"] = pd.to_numeric(final_jem_df["jem-depth"], errors='coerce').abs()
final_jem_df["lims-depth"] = pd.to_numeric(final_jem_df["lims-depth"], errors='coerce').abs()
final_jem_df["jem-pressure_extraction"] = pd.to_numeric(final_jem_df["jem-pressure_extraction"], errors='coerce').abs()
final_jem_df["jem-pressure_retraction"] = pd.to_numeric(final_jem_df["jem-pressure_retraction"], errors='coerce').abs()
final_jem_df["jem-nucleus_pipette_res"] = pd.to_numeric(final_jem_df["jem-nucleus_pipette_res"], errors='coerce').abs()
final_jem_df["jem-nucleus_end_seal_res"] = pd.to_numeric(final_jem_df["jem-nucleus_end_seal_res"], errors='coerce').abs()

# Create duration columns
final_jem_df["jem-time_duration_exp"] = pd.to_datetime(final_jem_df["jem-time_exp_retraction_end"]) - pd.to_datetime(final_jem_df["jem-time_exp_whole_cell_start"])
final_jem_df["jem-time_duration_ext"] = pd.to_datetime(final_jem_df["jem-time_exp_extraction_end"]) - pd.to_datetime(final_jem_df["jem-time_exp_extraction_start"])
final_jem_df["jem-time_duration_ret"] = pd.to_datetime(final_jem_df["jem-time_exp_retraction_end"]) - pd.to_datetime(final_jem_df["jem-time_exp_extraction_end"])
final_jem_df["jem-time_duration_exp"] = (final_jem_df["jem-time_duration_exp"].astype('timedelta64[s]'))/60
final_jem_df["jem-time_duration_ext"] = (final_jem_df["jem-time_duration_ext"].astype('timedelta64[s]'))/60
final_jem_df["jem-time_duration_ret"] = (final_jem_df["jem-time_duration_ret"].astype('timedelta64[s]'))/60

In [ ]:
# Test columns
final_jem_df["test-mismatch_depth"] = final_jem_df["jem-depth"] == final_jem_df["lims-depth"]
final_jem_df["test-mismatch_id_cell_specimen"] = final_jem_df["jem-id_cell_specimen"] == final_jem_df["lims-id_cell_specimen"]

In [ ]:
# Replace values in columns
final_jem_df["jem-region_major_minor"] = final_jem_df["jem-region_major_minor"].replace(to_replace="layer ", value="L", regex=True)
final_jem_df["jem-region_major_minor"] = final_jem_df["jem-region_major_minor"].replace(to_replace="/", value="-", regex=True)
final_jem_df["jem-region_minor"] = final_jem_df["jem-region_minor"].replace(to_replace="layer ", value="L", regex=True)
final_jem_df["jem-region_minor"] = final_jem_df["jem-region_minor"].replace(to_replace="/", value="-", regex=True)
final_jem_df["jem-health_cell"] = final_jem_df["jem-health_cell"].replace({"None": np.nan})
final_jem_df["jem-project_name"] = final_jem_df["jem-project_name"].replace({np.nan: "None"})
final_jem_df["jem-health_slice_initial"] = final_jem_df["jem-health_slice_initial"].replace({"Damaged": "Damage (Tissue Processing)",
                                                                                             "Good": "Healthy",
                                                                                             "Wave of Death": "Wave of Death (after 30 min)",
                                                                                             "'Wave of Death'": "Wave of Death (after 30 min)"})
final_jem_df["jem-status_reporter"] = final_jem_df["jem-status_reporter"].replace({"Cre+": "Positive",
                                                                                   "Cre-": "Negative",
                                                                                   "human": np.nan,
                                                                                   "None": np.nan})

In [ ]:
# Convert column to integer column
final_jem_df["jem-health_cell"] = final_jem_df["jem-health_cell"].fillna(value=0)
final_jem_df["jem-options_attempt"] = final_jem_df["jem-options_attempt"].fillna(value=0)
final_jem_df["jem-health_cell"] = final_jem_df["jem-health_cell"].astype(int)
final_jem_df["jem-options_attempt"] = final_jem_df["jem-options_attempt"].astype(int)
final_jem_df["jem-id_rig_number"] = final_jem_df["jem-id_rig_number"].astype(int)

In [ ]:
# Add a new column
final_jem_df["jem-nucleus_post_patch_detail"] = pd.np.where(((final_jem_df["jem-nucleus_post_patch"]=="nucleus_present")|(final_jem_df["jem-nucleus_post_patch"]=="entire_cell"))&(final_jem_df["jem-nucleus_end_seal_res"]>=1000), "Nuc-giga-seal",
                                                            pd.np.where(((final_jem_df["jem-nucleus_post_patch"]=="nucleus_present")|(final_jem_df["jem-nucleus_post_patch"]=="entire_cell"))&(final_jem_df["jem-nucleus_end_seal_res"]<1000), "Nuc-low-seal",
                                                            pd.np.where(final_jem_df["jem-nucleus_post_patch"]=="nucleus_absent", "No-seal",
                                                            pd.np.where(final_jem_df["jem-nucleus_post_patch"]=="unknown", "Unknown", "Not applicable"))))

In [ ]:
# Sort columns alphabetically and by date
final_jem_df = final_jem_df.sort_index(axis=1)
final_jem_df.sort_values(by=["jem-date_patch_y-m-d",
                             "jem-id_slice_specimen",
                             "jem-id_cell_specimen",
                             "jem-options_attempt"],
                        inplace=True)

In [ ]:
# Dropping columns
final_jem_df.drop(columns=["approach.pilotTest01",
                           "approach.pilotTest04",
                           "approach.pilotTest05",
                           "approach.pilotTestYN",
                           "badSweeps",
                           "experimentType",
                           "extraction.extractionObservations",
                           "extraction.sampleObservations",
                           "extraction.timeRetractionStart",
                           "failureCause",
                           "flipped",
                           "internalSolution.concentrationBiocytin",
                           "internalSolution.concentrationRnaseInhibitor", 
                           "internalSolution.version",
                           "internalSolution.volume",
                           "jem-health_slice",
                           "recording.accessR",
                           "recording.humanCellTypePrediction",
                           "recording.membraneV",
                           "recording.rheobase", 
                           "extraction.tubeID",
                           "jem_created",
                           "jem_date_dt",
                           "jem_date_m",
                           "extraction.pilotNameExtra",
                           "formVersion",
                           "jem-depth_current",
                           "jem-depth_old",
                           "jem-time_exp_end_old",
                           "jem-time_exp_retraction_end_current", 
                           "wellID"],
                  inplace=True)

##### Final Output

In [ ]:
# Dataframe to csvs
final_jem_df.to_csv("final_jem.csv", index=False)

# Dataframe to excel
final_jem_df.to_excel(excel_writer="final_jem.xlsx", index=False)

##### Testing

In [ ]:
len(final_jem_df)

In [ ]:
df = final_jem_df[final_jem_df["jem-status_success_fail"] == "SUCCESS"]
len(df)

In [ ]:
df1 = df[df["jem-id_patched_cell_container"] == "Not Applicable"]
len(df1)

In [ ]:
df2 = df1[df1["jem-id_cell_specimen"].isnull()]
len(df2)

In [ ]:
df['jem-options_attempt'].unique()

In [ ]:
df['jem-options_attempt'] = df['jem-options_attempt'].astype(str)
df['jem-options_attempt'] = df['jem-options_attempt'].str.zfill(2)

In [ ]:
df['jem-options_attempt'].unique()

In [ ]:
df2["jem-id_cell_specimen"] = df1["jem-id_slice_specimen"] + "." + df['jem-options_attempt']

In [ ]:
df2.sort_values(by=["jem-date_patch_y-m-d","jem-id_slice_specimen","jem-options_attempt"],inplace=True)